In [1]:
import pybamm 

In [2]:
options = {"thermal": "x-full"}
model = pybamm.lithium_ion.SPMe(options=options) 
parameter_values = model.default_parameter_values

In [3]:
import numpy as np

In [4]:
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.13699999999999998,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x00000218DF439040>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x00000218DF3298B0>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial concentration in positive electrode [mol.m-3]': 30730.755438556498,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'I

In [5]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0,10800])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0153495295262456, step=0.01015349529526245…

In [6]:
# defining array of C_rates
C_Rates = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,7,7.1,7.2,7.3,7.4,7.5,8,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,9,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10]


In [7]:

NSOC = []
PSOC = []
Temp = []
Voltage = []
C_Rate = []

n = len(C_Rates)


for i in range (n):
    sim = pybamm.Simulation(model, parameter_values=parameter_values, C_rate = C_Rates[i])
    sim.solve([0,10800])
    nsoc = sim.solution["Negative electrode SOC"].entries
    psoc = sim.solution["Positive electrode SOC"].entries
    temp = sim.solution["Negative electrode temperature [K]"].entries[-1]
    voltage = sim.solution["Terminal voltage [V]"].entries
    
    for j in range(len(nsoc)):
        C_Rate.append(C_Rates[i])
        NSOC.append(nsoc[j])
        Temp.append(temp[j])
        PSOC.append(psoc[j])
        Voltage.append(voltage[j])
    print(C_Rates[i])

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1
1.1
1.2
1.3
1.4
1.5
1.6
1.7
1.8
1.9
2
2.1
2.2
2.3
2.4
2.5
2.6
2.7
2.8
2.9
3
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
4
4.1
4.2
4.3
4.4
4.5
4.6
4.7
4.8
4.9
5
5.1
5.2
5.3
5.4
5.5
5.6
5.7
5.8
5.9
6
6.1
6.2
6.3
6.4
6.5
6.6
6.7
6.8
6.9
7
7.1
7.2
7.3
7.4
7.5
8
8.1
8.2
8.3
8.4
8.5
8.6
8.7
8.8
8.9
9
9.1
9.2
9.3
9.4
9.5
9.6
9.7
9.8
9.9
10


In [8]:
# creating a dataframe and storing the results

import pandas as pd

df = pd.DataFrame()
df['C_Rate'] = C_Rate
df['NSOC'] = NSOC
df['PSOC'] = PSOC
df['Temp'] = Temp
df['Voltage'] = Voltage

# storing result into spmt.csv file
df.to_csv(r"data.csv")
df.head()

,C_Rate,NSOC,PSOC,Temp,Voltage
0,0.1,0.800000,0.600000,298.150000,3.841727
1,0.1,0.798190,0.601060,298.162147,3.838888
2,0.1,0.796380,0.602119,298.174467,3.837526
3,0.1,0.794569,0.603179,298.186814,3.836223
4,0.1,0.792759,0.604238,298.199182,3.834928


In [9]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,5]], axis = 1)
y = df.drop(df.columns[[0,1,2,3,4]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)
print(x)
print(y)

      C_Rate      NSOC      PSOC        Temp
0        0.1  0.800000  0.600000  298.150000
1        0.1  0.798190  0.601060  298.162147
2        0.1  0.796380  0.602119  298.174467
3        0.1  0.794569  0.603179  298.186814
4        0.1  0.792759  0.604238  298.199182
...      ...       ...       ...         ...
1525    10.0  0.800000  0.600000  298.150000
1526    10.0  0.618975  0.705961  320.951827
1527    10.0  0.437950  0.811922  335.525786
1528    10.0  0.256926  0.917883  347.712462
1529    10.0  0.227144  0.935316  350.999780

[1530 rows x 4 columns]
       Voltage
0     3.841727
1     3.838888
2     3.837526
3     3.836223
4     3.834928
...        ...
1525  3.527892
1526  3.502478
1527  3.523391
1528  3.447120
1529  3.105347

[1530 rows x 1 columns]


In [14]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 4),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 200)


Epoch 1/200
39/39 [==============================] - 1s 730us/step - loss: 5.8692 - mean_squared_error: 5.8692
Epoch 2/200
39/39 [==============================] - 0s 842us/step - loss: 0.1604 - mean_squared_error: 0.1604
Epoch 3/200
39/39 [==============================] - 0s 842us/step - loss: 0.0957 - mean_squared_error: 0.0957
Epoch 4/200
39/39 [==============================] - 0s 842us/step - loss: 0.0838 - mean_squared_error: 0.0838
Epoch 5/200
39/39 [==============================] - 0s 843us/step - loss: 0.0860 - mean_squared_error: 0.0860
Epoch 6/200
39/39 [==============================] - 0s 841us/step - loss: 0.0727 - mean_squared_error: 0.0727
Epoch 7/200
39/39 [==============================] - 0s 841us/step - loss: 0.0689 - mean_squared_error: 0.0689
Epoch 8/200
39/39 [==============================] - 0s 842us/step - loss: 0.0672 - mean_squared_error: 0.0672
Epoch 9/200
39/39 [==============================] - 0s 843us/step - loss: 0.0774 - mean_squared_error: 0.0774
E

39/39 [==============================] - 0s 632us/step - loss: 0.0298 - mean_squared_error: 0.0298
Epoch 147/200
39/39 [==============================] - 0s 632us/step - loss: 0.0270 - mean_squared_error: 0.0270
Epoch 148/200
39/39 [==============================] - 0s 632us/step - loss: 0.0165 - mean_squared_error: 0.0165
Epoch 149/200
39/39 [==============================] - 0s 842us/step - loss: 0.0139 - mean_squared_error: 0.0139
Epoch 150/200
39/39 [==============================] - 0s 632us/step - loss: 0.0370 - mean_squared_error: 0.0370
Epoch 151/200
39/39 [==============================] - 0s 632us/step - loss: 0.0215 - mean_squared_error: 0.0215
Epoch 152/200
39/39 [==============================] - 0s 632us/step - loss: 0.0194 - mean_squared_error: 0.0194
Epoch 153/200
39/39 [==============================] - 0s 842us/step - loss: 0.0165 - mean_squared_error: 0.0165
Epoch 154/200
39/39 [==============================] - 0s 632us/step - loss: 0.0190 - mean_squared_error: 0.01

In [15]:
score = model.evaluate(x_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 0.0118 - mean_squared_error: 0.0118


In [16]:
df = df.drop(df.columns[[0]], axis = 1)

In [17]:
y_predicted = model.predict(x)
df['Predicted_Voltage'] = y_predicted
df.to_csv(r"data.csv")
df.head()

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861382
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010


## NSOC

In [22]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,2,5]], axis = 1)
y = df.drop(df.columns[[0,1,3,4,5,6]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

      C_Rate      PSOC        Temp  Predicted_Voltage
0        0.1  0.600000  298.150000           3.862841
1        0.1  0.601060  298.162147           3.861382
2        0.1  0.602119  298.174467           3.859925
3        0.1  0.603179  298.186814           3.858467
4        0.1  0.604238  298.199182           3.857010
...      ...       ...         ...                ...
1525    10.0  0.600000  298.150000           3.302495
1526    10.0  0.705961  320.951827           3.354240
1527    10.0  0.811922  335.525786           3.327200
1528    10.0  0.917883  347.712462           3.277515
1529    10.0  0.935316  350.999780           3.281504

[1530 rows x 4 columns]
          NSOC
0     0.800000
1     0.798190
2     0.796380
3     0.794569
4     0.792759
...        ...
1525  0.800000
1526  0.618975
1527  0.437950
1528  0.256926
1529  0.227144

[1530 rows x 1 columns]


In [23]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 4),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 150)


Epoch 1/150
39/39 [==============================] - 0s 631us/step - loss: 22.9668 - mean_squared_error: 22.9668
Epoch 2/150
39/39 [==============================] - 0s 842us/step - loss: 0.2912 - mean_squared_error: 0.2912
Epoch 3/150
39/39 [==============================] - 0s 632us/step - loss: 0.0887 - mean_squared_error: 0.0887
Epoch 4/150
39/39 [==============================] - 0s 842us/step - loss: 0.0643 - mean_squared_error: 0.0643
Epoch 5/150
39/39 [==============================] - 0s 842us/step - loss: 0.0566 - mean_squared_error: 0.0566
Epoch 6/150
39/39 [==============================] - 0s 842us/step - loss: 0.0529 - mean_squared_error: 0.0529
Epoch 7/150
39/39 [==============================] - 0s 842us/step - loss: 0.0493 - mean_squared_error: 0.0493
Epoch 8/150
39/39 [==============================] - 0s 842us/step - loss: 0.0449 - mean_squared_error: 0.0449
Epoch 9/150
39/39 [==============================] - 0s 842us/step - loss: 0.0408 - mean_squared_error: 0.0408

39/39 [==============================] - 0s 3ms/step - loss: 0.0137 - mean_squared_error: 0.0137
Epoch 150/150
39/39 [==============================] - 0s 3ms/step - loss: 0.0089 - mean_squared_error: 0.0089


In [25]:
df = df.drop(df.columns[[0]], axis = 1)

In [26]:
score = model.evaluate(x_test, y_test)
print(score)

10/10 [==============================] - 0s 885us/step - loss: 0.0031 - mean_squared_error: 0.0031
[0.0031097237952053547, 0.0031097237952053547]


In [27]:
nsoc_predicted = model.predict(x)
df['Predicted_NSOC'] = nsoc_predicted
df.head(5)

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861382,0.795102
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615


In [28]:
df.to_csv(r"data.csv")
df.head()

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861382,0.795102
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615


## PSOC

In [38]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,3,5,7]], axis = 1)
y = df.drop(df.columns[[0,1,2,4,5,6,7]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

      C_Rate      NSOC        Temp  Predicted_Voltage
0        0.1  0.800000  298.150000           3.862841
1        0.1  0.798190  298.162147           3.861381
2        0.1  0.796380  298.174467           3.859925
3        0.1  0.794569  298.186814           3.858467
4        0.1  0.792759  298.199182           3.857010
...      ...       ...         ...                ...
1525    10.0  0.800000  298.150000           3.302495
1526    10.0  0.618975  320.951827           3.354240
1527    10.0  0.437950  335.525786           3.327200
1528    10.0  0.256926  347.712462           3.277515
1529    10.0  0.227144  350.999780           3.281504

[1530 rows x 4 columns]
          PSOC
0     0.600000
1     0.601060
2     0.602119
3     0.603179
4     0.604238
...        ...
1525  0.600000
1526  0.705961
1527  0.811922
1528  0.917883
1529  0.935316

[1530 rows x 1 columns]


In [39]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 4),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 200)

Epoch 1/200
39/39 [==============================] - 1s 841us/step - loss: 131.6554 - mean_squared_error: 131.6554
Epoch 2/200
39/39 [==============================] - 0s 842us/step - loss: 1.0037 - mean_squared_error: 1.0037
Epoch 3/200
39/39 [==============================] - 0s 842us/step - loss: 0.0294 - mean_squared_error: 0.0294
Epoch 4/200
39/39 [==============================] - 0s 842us/step - loss: 0.0186 - mean_squared_error: 0.0186
Epoch 5/200
39/39 [==============================] - 0s 842us/step - loss: 0.0178 - mean_squared_error: 0.0178
Epoch 6/200
39/39 [==============================] - 0s 842us/step - loss: 0.0175 - mean_squared_error: 0.0175
Epoch 7/200
39/39 [==============================] - 0s 1ms/step - loss: 0.0169 - mean_squared_error: 0.0169
Epoch 8/200
39/39 [==============================] - 0s 842us/step - loss: 0.0156 - mean_squared_error: 0.0156
Epoch 9/200
39/39 [==============================] - 0s 1ms/step - loss: 0.0153 - mean_squared_error: 0.0153
E

39/39 [==============================] - 0s 632us/step - loss: 0.0101 - mean_squared_error: 0.0101  
Epoch 148/200
39/39 [==============================] - 0s 842us/step - loss: 0.0172 - mean_squared_error: 0.0172
Epoch 149/200
39/39 [==============================] - 0s 632us/step - loss: 0.0049 - mean_squared_error: 0.0049  
Epoch 150/200
39/39 [==============================] - 0s 842us/step - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 151/200
39/39 [==============================] - 0s 842us/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 152/200
39/39 [==============================] - 0s 843us/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 153/200
39/39 [==============================] - 0s 841us/step - loss: 0.0091 - mean_squared_error: 0.0091
Epoch 154/200
39/39 [==============================] - 0s 842us/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 155/200
39/39 [==============================] - 0s 632us/step - loss: 0.0068 - mean_squared_error: 

In [40]:
df = df.drop(df.columns[[0]], axis = 1)

In [41]:
score = model.evaluate(x_test, y_test)
print(score)

10/10 [==============================] - 0s 550us/step - loss: 0.0022 - mean_squared_error: 0.0022
[0.0021663850639015436, 0.0021663850639015436]


In [42]:
psoc_predicted = model.predict(x)
df['Predicted_PSOC'] = psoc_predicted
df.head(5)

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC,Predicted_PSOC
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604,0.588326
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861381,0.795102,0.589095
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607,0.589850
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109,0.590613
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615,0.591375


In [43]:
df.to_csv(r"data.csv")

## Temp

In [56]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,4,5,7,8]], axis = 1)
y = df.drop(df.columns[[0,1,2,3,5,6,7,8]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

      C_Rate      NSOC      PSOC  Predicted_Voltage
0        0.1  0.800000  0.600000           3.862841
1        0.1  0.798190  0.601060           3.861382
2        0.1  0.796380  0.602119           3.859925
3        0.1  0.794569  0.603179           3.858467
4        0.1  0.792759  0.604238           3.857010
...      ...       ...       ...                ...
1525    10.0  0.800000  0.600000           3.302495
1526    10.0  0.618975  0.705961           3.354240
1527    10.0  0.437950  0.811922           3.327200
1528    10.0  0.256926  0.917883           3.277515
1529    10.0  0.227144  0.935316           3.281504

[1530 rows x 4 columns]
            Temp
0     298.150000
1     298.162147
2     298.174467
3     298.186814
4     298.199182
...          ...
1525  298.150000
1526  320.951827
1527  335.525786
1528  347.712462
1529  350.999780

[1530 rows x 1 columns]


In [57]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 4),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 450)

Epoch 1/450
39/39 [==============================] - 0s 617us/step - loss: 95931.7070 - mean_squared_error: 95931.7070
Epoch 2/450
39/39 [==============================] - 0s 541us/step - loss: 87994.1910 - mean_squared_error: 87994.1910
Epoch 3/450
39/39 [==============================] - 0s 525us/step - loss: 60941.3682 - mean_squared_error: 60941.3682
Epoch 4/450
39/39 [==============================] - 0s 614us/step - loss: 17038.6150 - mean_squared_error: 17038.6150
Epoch 5/450
39/39 [==============================] - 0s 513us/step - loss: 4911.1744 - mean_squared_error: 4911.1744
Epoch 6/450
39/39 [==============================] - 0s 525us/step - loss: 3020.0189 - mean_squared_error: 3020.0189
Epoch 7/450
39/39 [==============================] - 0s 558us/step - loss: 1709.7618 - mean_squared_error: 1709.7618
Epoch 8/450
39/39 [==============================] - 0s 536us/step - loss: 1025.9784 - mean_squared_error: 1025.9784
Epoch 9/450
39/39 [==============================] - 0s 

39/39 [==============================] - 0s 472us/step - loss: 0.5866 - mean_squared_error: 0.5866
Epoch 145/450
39/39 [==============================] - 0s 472us/step - loss: 0.5447 - mean_squared_error: 0.5447
Epoch 146/450
39/39 [==============================] - 0s 541us/step - loss: 0.7102 - mean_squared_error: 0.7102
Epoch 147/450
39/39 [==============================] - 0s 472us/step - loss: 0.5338 - mean_squared_error: 0.5338
Epoch 148/450
39/39 [==============================] - 0s 472us/step - loss: 0.6958 - mean_squared_error: 0.6958
Epoch 149/450
39/39 [==============================] - 0s 499us/step - loss: 0.5744 - mean_squared_error: 0.5744
Epoch 150/450
39/39 [==============================] - 0s 525us/step - loss: 0.6385 - mean_squared_error: 0.6385
Epoch 151/450
39/39 [==============================] - 0s 499us/step - loss: 0.5567 - mean_squared_error: 0.5567
Epoch 152/450
39/39 [==============================] - 0s 499us/step - loss: 0.5056 - mean_squared_error: 0.50

39/39 [==============================] - 0s 499us/step - loss: 0.4606 - mean_squared_error: 0.4606
Epoch 289/450
39/39 [==============================] - 0s 525us/step - loss: 0.5007 - mean_squared_error: 0.5007
Epoch 290/450
39/39 [==============================] - 0s 499us/step - loss: 0.6901 - mean_squared_error: 0.6901
Epoch 291/450
39/39 [==============================] - 0s 525us/step - loss: 0.5985 - mean_squared_error: 0.5985
Epoch 292/450
39/39 [==============================] - 0s 499us/step - loss: 0.5552 - mean_squared_error: 0.5552
Epoch 293/450
39/39 [==============================] - 0s 499us/step - loss: 0.6203 - mean_squared_error: 0.6203
Epoch 294/450
39/39 [==============================] - 0s 472us/step - loss: 0.4996 - mean_squared_error: 0.4996
Epoch 295/450
39/39 [==============================] - 0s 499us/step - loss: 0.5333 - mean_squared_error: 0.5333
Epoch 296/450
39/39 [==============================] - 0s 525us/step - loss: 0.4259 - mean_squared_error: 0.42

39/39 [==============================] - 0s 499us/step - loss: 0.4356 - mean_squared_error: 0.4356
Epoch 433/450
39/39 [==============================] - 0s 525us/step - loss: 0.4139 - mean_squared_error: 0.4139
Epoch 434/450
39/39 [==============================] - 0s 499us/step - loss: 0.3534 - mean_squared_error: 0.3534
Epoch 435/450
39/39 [==============================] - 0s 499us/step - loss: 0.6858 - mean_squared_error: 0.6858
Epoch 436/450
39/39 [==============================] - 0s 472us/step - loss: 0.3408 - mean_squared_error: 0.3408
Epoch 437/450
39/39 [==============================] - 0s 525us/step - loss: 0.3763 - mean_squared_error: 0.3763
Epoch 438/450
39/39 [==============================] - 0s 551us/step - loss: 0.3969 - mean_squared_error: 0.3969
Epoch 439/450
39/39 [==============================] - 0s 556us/step - loss: 0.3790 - mean_squared_error: 0.3790
Epoch 440/450
39/39 [==============================] - 0s 569us/step - loss: 0.3941 - mean_squared_error: 0.39

In [58]:
df = df.drop(df.columns[[0]], axis = 1)

In [59]:
score = model.evaluate(x_test, y_test)
print(score)

10/10 [==============================] - 0s 2ms/step - loss: 0.3370 - mean_squared_error: 0.3370
[0.3370417654514313, 0.3370417654514313]


In [60]:
temp_predicted = model.predict(x)
df['Predicted_Temp'] = temp_predicted
df.head(5)

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC,Predicted_PSOC,Predicted_Temp
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604,0.588326,296.446136
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861382,0.795102,0.589095,296.486786
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607,0.589850,296.527557
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109,0.590613,296.568329
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615,0.591375,296.609161


In [61]:
df.to_csv(r"data.csv")

## C_Rate

In [71]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,1,5,7,8,9]], axis = 1)
y = df.drop(df.columns[[0,2,3,4,5,6,7,8,9]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

          NSOC      PSOC        Temp  Predicted_Voltage
0     0.800000  0.600000  298.150000           3.862841
1     0.798190  0.601060  298.162147           3.861381
2     0.796380  0.602119  298.174467           3.859925
3     0.794569  0.603179  298.186814           3.858467
4     0.792759  0.604238  298.199182           3.857010
...        ...       ...         ...                ...
1525  0.800000  0.600000  298.150000           3.302495
1526  0.618975  0.705961  320.951827           3.354240
1527  0.437950  0.811922  335.525786           3.327200
1528  0.256926  0.917883  347.712462           3.277515
1529  0.227144  0.935316  350.999780           3.281504

[1530 rows x 4 columns]
      C_Rate
0        0.1
1        0.1
2        0.1
3        0.1
4        0.1
...      ...
1525    10.0
1526    10.0
1527    10.0
1528    10.0
1529    10.0

[1530 rows x 1 columns]


In [72]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 4),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 800)

Epoch 1/800
39/39 [==============================] - 0s 840us/step - loss: 110.3813 - mean_squared_error: 110.3813
Epoch 2/800
39/39 [==============================] - 0s 614us/step - loss: 7.5197 - mean_squared_error: 7.5197
Epoch 3/800
39/39 [==============================] - 0s 551us/step - loss: 6.8841 - mean_squared_error: 6.8841
Epoch 4/800
39/39 [==============================] - 0s 551us/step - loss: 6.8679 - mean_squared_error: 6.8679
Epoch 5/800
39/39 [==============================] - 0s 525us/step - loss: 6.4899 - mean_squared_error: 6.4899
Epoch 6/800
39/39 [==============================] - 0s 536us/step - loss: 6.4903 - mean_squared_error: 6.4903
Epoch 7/800
39/39 [==============================] - 0s 551us/step - loss: 7.0291 - mean_squared_error: 7.0291
Epoch 8/800
39/39 [==============================] - 0s 499us/step - loss: 6.9323 - mean_squared_error: 6.9323
Epoch 9/800
39/39 [==============================] - 0s 499us/step - loss: 7.1470 - mean_squared_error: 7.14

39/39 [==============================] - 0s 499us/step - loss: 5.0990 - mean_squared_error: 5.0990
Epoch 147/800
39/39 [==============================] - 0s 551us/step - loss: 5.0057 - mean_squared_error: 5.0057
Epoch 148/800
39/39 [==============================] - 0s 518us/step - loss: 4.1226 - mean_squared_error: 4.1226
Epoch 149/800
39/39 [==============================] - 0s 525us/step - loss: 4.1510 - mean_squared_error: 4.1510
Epoch 150/800
39/39 [==============================] - 0s 525us/step - loss: 3.8244 - mean_squared_error: 3.8244
Epoch 151/800
39/39 [==============================] - 0s 472us/step - loss: 5.1719 - mean_squared_error: 5.1719
Epoch 152/800
39/39 [==============================] - 0s 499us/step - loss: 4.2415 - mean_squared_error: 4.2415
Epoch 153/800
39/39 [==============================] - 0s 472us/step - loss: 4.1971 - mean_squared_error: 4.1971
Epoch 154/800
39/39 [==============================] - 0s 499us/step - loss: 4.4142 - mean_squared_error: 4.41

39/39 [==============================] - 0s 472us/step - loss: 0.1574 - mean_squared_error: 0.1574
Epoch 291/800
39/39 [==============================] - 0s 472us/step - loss: 0.1372 - mean_squared_error: 0.1372
Epoch 292/800
39/39 [==============================] - 0s 499us/step - loss: 0.3317 - mean_squared_error: 0.3317
Epoch 293/800
39/39 [==============================] - 0s 525us/step - loss: 0.2504 - mean_squared_error: 0.2504
Epoch 294/800
39/39 [==============================] - 0s 525us/step - loss: 0.2699 - mean_squared_error: 0.2699
Epoch 295/800
39/39 [==============================] - 0s 551us/step - loss: 0.4777 - mean_squared_error: 0.4777
Epoch 296/800
39/39 [==============================] - 0s 551us/step - loss: 0.1942 - mean_squared_error: 0.1942
Epoch 297/800
39/39 [==============================] - 0s 550us/step - loss: 0.5460 - mean_squared_error: 0.5460
Epoch 298/800
39/39 [==============================] - 0s 603us/step - loss: 0.5809 - mean_squared_error: 0.58

39/39 [==============================] - 0s 502us/step - loss: 0.0825 - mean_squared_error: 0.0825
Epoch 435/800
39/39 [==============================] - 0s 525us/step - loss: 0.1021 - mean_squared_error: 0.1021
Epoch 436/800
39/39 [==============================] - 0s 499us/step - loss: 0.0886 - mean_squared_error: 0.0886
Epoch 437/800
39/39 [==============================] - 0s 472us/step - loss: 0.0667 - mean_squared_error: 0.0667
Epoch 438/800
39/39 [==============================] - 0s 499us/step - loss: 0.0957 - mean_squared_error: 0.0957
Epoch 439/800
39/39 [==============================] - 0s 551us/step - loss: 0.1610 - mean_squared_error: 0.1610
Epoch 440/800
39/39 [==============================] - 0s 499us/step - loss: 0.3716 - mean_squared_error: 0.3716
Epoch 441/800
39/39 [==============================] - 0s 499us/step - loss: 0.2558 - mean_squared_error: 0.2558
Epoch 442/800
39/39 [==============================] - 0s 472us/step - loss: 0.0976 - mean_squared_error: 0.09

39/39 [==============================] - 0s 472us/step - loss: 0.1155 - mean_squared_error: 0.1155
Epoch 579/800
39/39 [==============================] - 0s 525us/step - loss: 0.0882 - mean_squared_error: 0.0882
Epoch 580/800
39/39 [==============================] - 0s 499us/step - loss: 0.0283 - mean_squared_error: 0.0283
Epoch 581/800
39/39 [==============================] - 0s 499us/step - loss: 0.0447 - mean_squared_error: 0.0447
Epoch 582/800
39/39 [==============================] - 0s 499us/step - loss: 0.2004 - mean_squared_error: 0.2004
Epoch 583/800
39/39 [==============================] - 0s 499us/step - loss: 0.0660 - mean_squared_error: 0.0660
Epoch 584/800
39/39 [==============================] - 0s 499us/step - loss: 0.0651 - mean_squared_error: 0.0651
Epoch 585/800
39/39 [==============================] - 0s 499us/step - loss: 0.0758 - mean_squared_error: 0.0758
Epoch 586/800
39/39 [==============================] - 0s 472us/step - loss: 0.0534 - mean_squared_error: 0.05

39/39 [==============================] - 0s 551us/step - loss: 0.0577 - mean_squared_error: 0.0577
Epoch 723/800
39/39 [==============================] - 0s 499us/step - loss: 0.0876 - mean_squared_error: 0.0876
Epoch 724/800
39/39 [==============================] - 0s 525us/step - loss: 0.0441 - mean_squared_error: 0.0441
Epoch 725/800
39/39 [==============================] - 0s 499us/step - loss: 0.0620 - mean_squared_error: 0.0620
Epoch 726/800
39/39 [==============================] - 0s 499us/step - loss: 0.1067 - mean_squared_error: 0.1067
Epoch 727/800
39/39 [==============================] - 0s 499us/step - loss: 0.0226 - mean_squared_error: 0.0226
Epoch 728/800
39/39 [==============================] - 0s 499us/step - loss: 0.0196 - mean_squared_error: 0.0196
Epoch 729/800
39/39 [==============================] - 0s 499us/step - loss: 0.0741 - mean_squared_error: 0.0741
Epoch 730/800
39/39 [==============================] - 0s 499us/step - loss: 0.0792 - mean_squared_error: 0.07

In [73]:
df = df.drop(df.columns[[0]], axis = 1)

In [74]:
score = model.evaluate(x_test, y_test)
print(score)

10/10 [==============================] - 0s 985us/step - loss: 0.1019 - mean_squared_error: 0.1019
[0.10186542570590973, 0.10186542570590973]


In [77]:
crate_predicted = model.predict(x)
df['Predicted_C_Rate'] = crate_predicted
df.head()

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC,Predicted_PSOC,Predicted_Temp,Predicted_C_Rate
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604,0.588326,296.44614,0.617810
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861381,0.795102,0.589095,296.48680,0.613803
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607,0.589850,296.52756,0.609785
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109,0.590613,296.56833,0.605798
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615,0.591375,296.60916,0.601786


In [78]:
df.to_csv(r"data.csv")

In [79]:
df

,C_Rate,NSOC,PSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC,Predicted_PSOC,Predicted_Temp,Predicted_C_Rate
0,0.1,0.800000,0.600000,298.150000,3.841727,3.862841,0.796604,0.588326,296.44614,0.617810
1,0.1,0.798190,0.601060,298.162147,3.838888,3.861381,0.795102,0.589095,296.48680,0.613803
2,0.1,0.796380,0.602119,298.174467,3.837526,3.859925,0.793607,0.589850,296.52756,0.609785
3,0.1,0.794569,0.603179,298.186814,3.836223,3.858467,0.792109,0.590613,296.56833,0.605798
4,0.1,0.792759,0.604238,298.199182,3.834928,3.857010,0.790615,0.591375,296.60916,0.601786
...,...,...,...,...,...,...,...,...,...,...
1525,10.0,0.800000,0.600000,298.150000,3.527892,3.302495,0.790725,0.486605,299.16330,9.768827
1526,10.0,0.618975,0.705961,320.951827,3.502478,3.354240,0.649714,0.639492,319.36765,10.328352
1527,10.0,0.437950,0.811922,335.525786,3.523391,3.327200,0.503786,0.762868,334.30075,10.408581
1528,10.0,0.256926,0.917883,347.712462,3.447120,3.277515,0.356508,0.877908,347.71878,10.347223
